<h1 style="font-size:4rem;color:orange;"> Weather Alert </h1>

## This program checks the weather at a specific location and sends a push notification at 6 am if it's below 30° F, snowing, or raining. With this alert, you'll be prepared for the weather ahead on the days it matters most!

## Project Setup

1) Create an account with **OpenWeatherMap** and generate an API Key. This enables you to access weather data from OpenWeatherMap. 
2) Create an account with **Pushbullet**, so you can send push notifications to your phone. Note that you must install the Pushbullet app on your phone and enable notification access. 

## Imports

In [ ]:
import requests 
import schedule 
import time as tm 
from pushbullet import Pushbullet 

# Weather Data Collection

Prior to requesting weather data from the OpenWeatherMap API, specify the url an API request should be sent to. This url consists of a base url, Weather API Key, and city. **Please edit "WEATHER_API_KEY" with your OpenWeather Map API Key and "CITY" with any location you choose.**

In [ ]:
BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"
WEATHER_API_KEY = ""
CITY = "New York City"
url = BASE_URL + "appid=" + WEATHER_API_KEY + "&q=" + CITY

### Weather Function

The **weather** function sends an API request to the url and stores the weather data it receives as a dictionary in the variable response.

In [ ]:
def weather():
    response = requests.get(url).json()     
    return response

Let's see what the data looks like by calling the weather function.

In [ ]:
weather()

We can see that information such as coordinates, wind speed, and temperatures are stored in a dictionary. However, the temperatures are in Kelvin and should be converted to Fahrenheit, so we can set a threshold of 30°F later on. 

### Kelvin to Fahrenheit Function

The function **kelvin_to_fahrenheit** converts temperatures from Kelvin to Fahrenheit based on the formula below

$$ Fahrenheit = (Kelvin - 273.15) \times \frac{9}{5} + 32 $$

In [ ]:
def kelvin_to_fahrenheit(kelvin):
    fahrenheit = (kelvin-273.15) * (9/5) + 32
    return fahrenheit

Here's an example of the 300 Kelvin being converted to 80.33° F

In [ ]:
kelvin_to_fahrenheit(300)

# Push Notification System 

Record your specific Pushbullet API Key in "PUSHBULLET_API_KEY"

In [ ]:
PUSHBULLET_API_KEY = ""

The **send_notification** function send a push notification to your phone with a title, message, and image using Pushubullet

In [ ]:
def send_notification(title,message,image_filename):
    pb = Pushbullet(PUSHBULLET_API_KEY)
    with open(image_filename, 'rb') as f:
        file_data = pb.upload_file(f, image_filename)
    pb.push_file(**file_data,title=title,body=message)

# Checking for Weather that is below 30° F, Snowing, or Raining

The function **check_weather_conditions** is designed to monitor the weather conditions and send tailored notifications based on certain criteria (snowing, raining, below 30° F).

**6 possible qualifying weather conditions** are checked to determine the appropriate message and image to send: 
1) If the temperature is below or equal to 30°F and it's snowing
2) If the temperature is below or equal to 30°F and it's raining
3) If the temperature is below or equal to 30°F
4) If it snowing
5) If it is raining
6) If the temperature is something else that doesn't meet the criteria above

In [ ]:
def check_weather_conditions():
    data = weather()
    temp_kelvin = data['main']['temp']
    temp_fahrenheit = kelvin_to_fahrenheit(temp_kelvin)
    description = data['weather'][0]['description']
    weather_description_rain = ['shower rain', 'rain', 'thunderstorm', 'thunderstorm with light rain', 'thunderstorm with rain', 'thunderstorm with heavy rain', 'light thunderstorm', 'heavy thunderstorm', 'ragged thunderstorm', 'thunderstorm with light drizzle', 'thunderstorm with drizzle', 'thunderstorm with heavy drizzle', 'light rain', 'moderate rain', 'heavy intensity rain', 'very heavy rain', 'extreme rain', 'freezing rain', 'light intensity shower rain', 'shower rain', 'heavy intensity shower rain', 'ragged shower rain' ]
    weather_description_snow = ['light snow', 'snow', 'heavy snow', 'sleet', 'light shower sleet', 'shower sleet', 'light rain and snow', 'rain and snow', 'light shower snow', 'shower snow', 'heavy shower snow']

    if temp_fahrenheit <=30 and description.lower() in weather_description_snow:
        message = f"It's below 30 degrees today and snowing! Who needs a freezer when you have the great outdoors?"
        send_notification("Weather Alert:", message,"Snow_and_Below30.jfif")
    elif temp_fahrenheit <=30 and description.lower() in weather_description_rain:
        message = f"It's below 30 degrees today and raining! We're getting free ice kisses from the sky."
        send_notification("Weather Alert:", message, "Rain_and_Below30.jpg")

    elif temp_fahrenheit <= 30:
        message = f"It's below 30 degrees today - Bundle up unless you want to be a human icicle!"
        send_notification("Weather Alert:", message, "Below30.jpg")

    elif description.lower() in weather_description_snow:
        message = f"Snow, snow, and more snow! Looks like mother nature's dandruff is acting up again!"
        send_notification("Weather Alert:", message, "snow.jpg")

    elif description.lower() in weather_description_rain:
        message = f"Don't forget an umbrella it's raining today!"
        send_notification("Weather Alert:", message, "rain.jpg")
   
    else:
        pass
    

Below are some sample push notifications if it was snowing, raining, or below 30°F. Note these messages and images are intended to be funny in contrast to the weather.

<img src="Push Notifications.png" alt="Image Description" style="width: 1000px;"/>

# Scheduling the program to run every day at 6 am

The **schedule library** runs the check_weather_conditions function everyday at 6 am in order to automate the process of monitoring the weather and sending push notifications accordingly. Note that this code is designed to run on a local machine and can only work while your laptop is on. For continous execution of this script offline, it would need to be deployed to a cloud based service such as AWS Lambda, Google Cloud Functions, or Azure functions.  

In [ ]:
schedule.every().day.at("06:00").do(check_weather_conditions)

while True:
    schedule.run_pending()
    tm.sleep(1)

# Conclusion

This project creates a functional weather alert system that provides notifications based on if it's raining, snowing, or freezing early in the morning. It helps you stay informed about changing weather conditions and helps you prepare for the weather ahead. This alert system can be refined even further by integrating weather data from multiple sources to improve accuracy or by allowing the user to customize the weather criteria on a more user friendly interface.